In [1]:
# To get onnx file, run in terminal:
# python setup.py test --addopts "-k test_brevitas_quartznet" 
import numpy as np
from finn.util.visualization import showInNetron
import onnx
from finn.core.modelwrapper import ModelWrapper

file_name = '/tmp/quartznet.onnx'
showInNetron(file_name)

Serving '/tmp/quartznet.onnx' at http://0.0.0.0:8081


In [2]:
from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from finn.transformation.general import RemoveUnusedTensors
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.infer_datatypes import InferDataTypes
#from finn.util.basic import *
#from finn.custom_op import *
from onnx import helper
#import onnx

model = ModelWrapper(file_name)
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferShapes())
model = model.transform(InferDataTypes())
model = model.transform(RemoveUnusedTensors())
model.save("/tmp/quartznet_modified.onnx")
showInNetron("/tmp/quartznet_modified.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/quartznet_modified.onnx' at http://0.0.0.0:8081


# Testing quartznet before and after transform

In [2]:
model = ModelWrapper("/tmp/quartznet_modified.onnx")

from finn.transformation.change_3d_tensors_to_4d import Change3DTo4DTensors

model = model.transform(Change3DTo4DTensors())
model.save("/tmp/quartznet_4d.onnx")
showInNetron("/tmp/quartznet_4d.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/quartznet_4d.onnx' at http://0.0.0.0:8081


In [2]:
import finn.core.onnx_exec as oxe

def make_unit_input(model):
    """ Creates numpy array that matches the input tensor shape """
    i_shape = []
    input_node = model.graph.input[0]
    input_node_shape = input_node.type.tensor_type.shape.dim
    input_node_name = input_node.name

    for d in input_node_shape:
        i_shape.append(d.dim_value)

    i_val = np.ones(i_shape).astype(np.float32)

    input_dict = {input_node_name: i_val}
    
    return input_dict

def execute_unit_input_test(model):
    """ Executes unit input test on the supplied model. Return value is the value of the output of the graph"""
    output_node_name = model.graph.output[0].name
    input_dict = make_unit_input(model)
    output_dict = oxe.execute_onnx(model, input_dict)
    expected = output_dict[output_node_name]
    
    return expected


In [ ]:
model_unmodified = ModelWrapper("/tmp/quartznet.onnx")
expected_unmodified = execute_unit_input_test(model_unmodified)
print("{}".format(expected_unmodified))

model_modified = ModelWrapper("/tmp/quartznet_4d.onnx")
expected_modified = execute_unit_input_test(model_modified)
print("{}".format(expected_modified))

assert (expected_unmodified==expected_modified).all()

# Creating the model

In [10]:
import numpy as np
from finn.util.visualization import showInNetron
from finn.core.modelwrapper import ModelWrapper

from finn.util.basic import *
import onnx

Mul1_node = onnx.helper.make_node(
    "Mul",
    inputs=['in1_mul1', 'in2_mul1'], #inputs
    outputs=['out1_mul1'], #outputs
    name='Mul1' #name
)

Conv1_node = onnx.helper.make_node(
    "Conv",
    inputs=['out1_mul1', 'in2_conv1'],
    outputs=['out1_conv1'],
    name='Conv1',
    dilations=[1],
    group=1,
    kernel_shape=[1],
    pads=[0,0],
    strides=[1]
)

Add1_node = onnx.helper.make_node(
    "Add",
    inputs=['out1_conv1', 'in2_add1'],
    outputs=['out1_add1'],
    name='Add1'
)

Mul2_node = onnx.helper.make_node(
    "Mul",
    inputs=['out1_add1', 'in2_mul2'],
    outputs=['out1_mul2'],
    name='Mul2'
)

Transpose1_node = onnx.helper.make_node(
    "Transpose",
    inputs=['out1_mul2'],
    outputs=['out1_transpose1'],
    name='Transpose1',
    perm=[0,2,1]
)

LogSoftmax1_node = onnx.helper.make_node(
    "LogSoftmax",
    inputs=['out1_transpose1'],
    outputs=['out1_logsoftmax1'],
    name='LogSoftmax1',
    axis=2
)

ArgMax1_node = onnx.helper.make_node(
    "ArgMax",
    inputs=['out1_logsoftmax1'],
    outputs=['out1_argmax1'],
    name='ArgMax1',
    axis=-1,
    keepdims=0
)

# Inputs and outputs
in1_mul1 = onnx.helper.make_tensor_value_info("in1_mul1", onnx.TensorProto.FLOAT, [1, 1024, 128])
#in2_mul1 = onnx.helper.make_tensor_value_info("in2_mul1", onnx.TensorProto.FLOAT, [1])
out1_argmax1 = onnx.helper.make_tensor_value_info("out1_argmax1", onnx.TensorProto.INT64, [1, 128])

# Value infos
out1_mul1 = onnx.helper.make_tensor_value_info("out1_mul1", onnx.TensorProto.FLOAT, [1, 1024, 128])
out1_conv1 = onnx.helper.make_tensor_value_info("out1_conv1", onnx.TensorProto.FLOAT, [1,29,128])
out1_add1 = onnx.helper.make_tensor_value_info("out1_add1", onnx.TensorProto.FLOAT, [1,29,128])
out1_mul2 = onnx.helper.make_tensor_value_info("out1_mul2", onnx.TensorProto.FLOAT, [1,29,128])
out1_transpose1 = onnx.helper.make_tensor_value_info("out1_transpose1", onnx.TensorProto.FLOAT, [1,128,29])
out1_logsoftmax1 = onnx.helper.make_tensor_value_info("out1_logsoftmax1", onnx.TensorProto.FLOAT, [1,128,29])

# Initializers
in2_mul1 = onnx.helper.make_tensor_value_info("in2_mul1", onnx.TensorProto.FLOAT, [1])
in2_conv1 = onnx.helper.make_tensor_value_info("in2_conv1", onnx.TensorProto.FLOAT, [29,1024,1])
in2_add1 = onnx.helper.make_tensor_value_info("in2_add1", onnx.TensorProto.FLOAT,[1,29,1])
in2_mul2 = onnx.helper.make_tensor_value_info("in2_mul2", onnx.TensorProto.FLOAT, [1])
           
#initializer1_name = 'in2_mul1'
#initializer1_shape = [1]
#init_value1 = gen_finn_dt_tensor(DataType.FLOAT32, initializer1_shape)
#model.set_initializer(initializer1_name, init_value1)
#initializer2_name = 'in2_conv1'
#initializer2_shape = [29,1024,1]
#init_value2 = gen_finn_dt_tensor(DataType.FLOAT32, initializer2_shape)
#model.set_initializer(initializer2_name, init_value2)
#initializer3_name = 'in2_add1'
#initializer3_shape = [1,29,1]
#init_value3 = gen_finn_dt_tensor(DataType.FLOAT32, initializer3_shape)
#model.set_initializer(initializer3_name, init_value3)
#initializer4_name = 'in2_mul2'
#initializer4_shape = [1]
#init_value4 = gen_finn_dt_tensor(DataType.FLOAT32, initializer4_shape)
#model.set_initializer(initializer4_name, init_value4)
    
graph = onnx.helper.make_graph(
    nodes=[
        Mul1_node,
        Conv1_node,
        Add1_node,
        Mul2_node,
        Transpose1_node,
        LogSoftmax1_node,
        ArgMax1_node
    ],
    name="4d_conversion_test_graph",
    inputs=[in1_mul1],
    outputs=[out1_argmax1],
    value_info=[
        out1_mul1,
        out1_conv1,
        out1_add1,
        out1_mul2,
        out1_transpose1,
        out1_logsoftmax1,
        in2_mul1,
        in2_conv1,
        in2_add1,
        in2_mul2
    ]
)

#onnx_model = onnx.helper.make_model(graph, producer_name="4d_conversion_test-model")
#onnx.save(onnx_model, "/tmp/4d_conversion_test_model.onnx")
#showInNetron("/tmp/4d_conversion_test_model.onnx")

#model = ModelWrapper("/tmp/4d_conversion_test_model.onnx")
onnx_model = onnx.helper.make_model(graph, producer_name="4d_conversion_test-model")
model = ModelWrapper(onnx_model)


# Inputs
def generate_random_input(model):
    """ Creates numpy array that matches the input tensor shape """
    i_shape = []
    input_dict={}
    for i in range(len(model.graph.input)):
        input_node = model.graph.input[i]
        input_node_name = input_node.name
        input_node_shape = model.get_tensor_shape(input_node_name)

        i_val = gen_finn_dt_tensor(DataType.FLOAT32, input_node_shape)
        #i_val = np.ones(i_shape).astype(np.float32)
        input_dict[input_node_name] = i_val
    return input_dict
input_dict = generate_random_input(model)

# Initializers
def set_all_initializers(model):
    for n in model.graph.node:
        if len(n.input)>1:            
            init_name = n.input[1]
            init_shape = model.get_tensor_shape(init_name)
            init_val = gen_finn_dt_tensor(DataType.FLOAT32, init_shape)
            model.set_initializer(init_name, init_val)
set_all_initializers(model)

onnx.save(onnx_model, "/tmp/4d_conversion_test_model.onnx")
showInNetron("/tmp/4d_conversion_test_model.onnx")



Stopping http://0.0.0.0:8081
Serving '/tmp/4d_conversion_test_model.onnx' at http://0.0.0.0:8081


In [11]:
import finn.core.onnx_exec as oxe

model = ModelWrapper("/tmp/4d_conversion_test_model.onnx")
showInNetron("/tmp/4d_conversion_test_model.onnx")
output_node_name = model.graph.output[0].name
output_dict = oxe.execute_onnx(model, input_dict, return_full_exec_context=True)
expected = output_dict[output_node_name]

print("{}\n{}".format(np.shape(expected),expected))



Stopping http://0.0.0.0:8081
Serving '/tmp/4d_conversion_test_model.onnx' at http://0.0.0.0:8081
(1, 128)
[[ 1 11  2 28  1  5  7 15 15 10  3 20 24  2 12 20 11 26  8 20 10 10 20  7
  17 18  4  7 27 15 13 23  1 14 23 15 28 23 24  3  9  4 12 16 22 23  4  7
  16 26  1 28 25 10  0  4  2 10 15 18 20  7  0 28 22  4 20 15 11 18 21  3
  15 11  9 23 25 15  3 27 16  0  2 28  7 20 11 20 13 10 14  2  7 19  5 23
  14 11 17 13 18 14  4 10 15 15 16 25 14  5 11  4 15 18 26  7  4  3 20 13
   6  5  7  6 18 14  8 21]]


In [12]:
from finn.transformation.change_3d_tensors_to_4d import Change3DTo4DTensors

model = ModelWrapper("/tmp/4d_conversion_test_model.onnx")
model = model.transform(Change3DTo4DTensors())
model.save("/tmp/4d_conversion_test_model_transformed.onnx")
showInNetron("/tmp/4d_conversion_test_model_transformed.onnx")

for k,v in input_dict.items():
    old_in_name = k
    old_in_val = v
    old_shape = np.shape(v)
    new_in_name = model.graph.input[0].name
    new_shape = old_shape + (1,)

new_in_val = np.reshape(v, new_shape)
del input_dict[old_in_name]
input_dict[new_in_name] = new_in_val

output_node_name = model.graph.output[0].name
output_dict = oxe.execute_onnx(model, input_dict, return_full_exec_context=True)
expected2 = output_dict[output_node_name]

print("{}\n{}".format(np.shape(expected2),expected2))



Stopping http://0.0.0.0:8081
Serving '/tmp/4d_conversion_test_model_transformed.onnx' at http://0.0.0.0:8081
(1, 128, 1)
[[[ 1]
  [11]
  [ 2]
  [28]
  [ 1]
  [ 5]
  [ 7]
  [15]
  [15]
  [10]
  [ 3]
  [20]
  [24]
  [ 2]
  [12]
  [20]
  [11]
  [26]
  [ 8]
  [20]
  [10]
  [10]
  [20]
  [ 7]
  [17]
  [18]
  [ 4]
  [ 7]
  [27]
  [15]
  [13]
  [23]
  [ 1]
  [14]
  [23]
  [15]
  [28]
  [23]
  [24]
  [ 3]
  [ 9]
  [ 4]
  [12]
  [16]
  [22]
  [23]
  [ 4]
  [ 7]
  [16]
  [26]
  [ 1]
  [28]
  [25]
  [10]
  [ 0]
  [ 4]
  [ 2]
  [10]
  [15]
  [18]
  [20]
  [ 7]
  [ 0]
  [28]
  [22]
  [ 4]
  [20]
  [15]
  [11]
  [18]
  [21]
  [ 3]
  [15]
  [11]
  [ 9]
  [23]
  [25]
  [15]
  [ 3]
  [27]
  [16]
  [ 0]
  [ 2]
  [28]
  [ 7]
  [20]
  [11]
  [20]
  [13]
  [10]
  [14]
  [ 2]
  [ 7]
  [19]
  [ 5]
  [23]
  [14]
  [11]
  [17]
  [13]
  [18]
  [14]
  [ 4]
  [10]
  [15]
  [15]
  [16]
  [25]
  [14]
  [ 5]
  [11]
  [ 4]
  [15]
  [18]
  [26]
  [ 7]
  [ 4]
  [ 3]
  [20]
  [13]
  [ 6]
  [ 5]
  [ 7]
  [ 6]
  [18]
  [

In [15]:
expected3 = np.reshape(expected2, np.shape(expected))
print((expected==expected3).all())

#assert(expected==expected2).all()

#model.save("/tmp/4d_conversion_test_model_transformed.onnx")
#showInNetron("/tmp/4d_conversion_test_model_transformed.onnx")

True


# Revisited version

In [28]:
import numpy as np
import onnx

import finn.core.onnx_exec as oxe
from finn.core.datatype import DataType
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.change_3d_tensors_to_4d import Change3DTo4DTensors
from finn.util.basic import gen_finn_dt_tensor


def generate_random_input(model):
    """
    Creates input dictionary with a random numpy array
    that matches the input tensor shape.
    """
    input_dict = {}
    for i in range(len(model.graph.input)):
        input_node = model.graph.input[i]
        input_node_name = input_node.name
        input_node_shape = model.get_tensor_shape(input_node_name)
        i_val = gen_finn_dt_tensor(DataType.FLOAT32, input_node_shape)
        input_dict[input_node_name] = i_val
    return input_dict


def set_all_initializers(model):
    """ Sets all initializers of the graph to a random value. """
    for n in model.graph.node:
        if len(n.input) > 1:
            init_name = n.input[1]
            init_shape = model.get_tensor_shape(init_name)
            init_val = gen_finn_dt_tensor(DataType.FLOAT32, init_shape)
            model.set_initializer(init_name, init_val)


def create_arbitrary_model(invalid=False):
    """
    Creates arbitrary model for testing the 3D to 4D transform.
    This model is based on a subpart of QuartzNet.
    """
    
    Mul1_node = onnx.helper.make_node(
        "Mul",
        inputs=["in1_mul1", "in2_mul1"],  # inputs
        outputs=["out1_mul1"],  # outputs
        name="Mul1",  # name
    )

    Conv1_node = onnx.helper.make_node(
        "Conv",
        inputs=["out1_mul1", "in2_conv1"],
        outputs=["out1_conv1"],
        name="Conv1",
        dilations=[1],
        group=1,
        kernel_shape=[1],
        pads=[0, 0],
        strides=[1],
    )
    
    if invalid is True: # To make the graph invalid, a ReLU node is added after the Conv node
        Relu1_node = onnx.helper.make_node(
            "Relu", inputs=["out1_conv1"], outputs=["out1_relu1"], name="Relu1"
        )
        Add1_node = onnx.helper.make_node(
            "Add", inputs=["out1_relu1", "in2_add1"], outputs=["out1_add1"], name="Add1"
        )
    else:
        Add1_node = onnx.helper.make_node(
            "Add", inputs=["out1_conv1", "in2_add1"], outputs=["out1_add1"], name="Add1"
        )

    Mul2_node = onnx.helper.make_node(
        "Mul", inputs=["out1_add1", "in2_mul2"], outputs=["out1_mul2"], name="Mul2"
    )

    Transpose1_node = onnx.helper.make_node(
        "Transpose",
        inputs=["out1_mul2"],
        outputs=["out1_transpose1"],
        name="Transpose1",
        perm=[0, 2, 1],
    )

    LogSoftmax1_node = onnx.helper.make_node(
        "LogSoftmax",
        inputs=["out1_transpose1"],
        outputs=["out1_logsoftmax1"],
        name="LogSoftmax1",
        axis=2,
    )

    ArgMax1_node = onnx.helper.make_node(
        "ArgMax",
        inputs=["out1_logsoftmax1"],
        outputs=["out1_argmax1"],
        name="ArgMax1",
        axis=-1,
        keepdims=0,
    )

    # Inputs and outputs
    in1_mul1 = onnx.helper.make_tensor_value_info(
        "in1_mul1", onnx.TensorProto.FLOAT, [1, 1024, 128]
    )
    out1_argmax1 = onnx.helper.make_tensor_value_info(
        "out1_argmax1", onnx.TensorProto.INT64, [1, 128]
    )

    # Value infos
    out1_mul1 = onnx.helper.make_tensor_value_info(
        "out1_mul1", onnx.TensorProto.FLOAT, [1, 1024, 128]
    )
    out1_conv1 = onnx.helper.make_tensor_value_info(
        "out1_conv1", onnx.TensorProto.FLOAT, [1, 29, 128]
    )
    
    if invalid is True:
        out1_relu1 = onnx.helper.make_tensor_value_info(
            "out1_relu1", onnx.TensorProto.FLOAT, [1, 29, 128]
        )

    out1_add1 = onnx.helper.make_tensor_value_info(
        "out1_add1", onnx.TensorProto.FLOAT, [1, 29, 128]
    )
            
    out1_mul2 = onnx.helper.make_tensor_value_info(
        "out1_mul2", onnx.TensorProto.FLOAT, [1, 29, 128]
    )
    out1_transpose1 = onnx.helper.make_tensor_value_info(
        "out1_transpose1", onnx.TensorProto.FLOAT, [1, 128, 29]
    )
    out1_logsoftmax1 = onnx.helper.make_tensor_value_info(
        "out1_logsoftmax1", onnx.TensorProto.FLOAT, [1, 128, 29]
    )

    # Initializers
    in2_mul1 = onnx.helper.make_tensor_value_info(
        "in2_mul1", onnx.TensorProto.FLOAT, [1]
    )
    in2_conv1 = onnx.helper.make_tensor_value_info(
        "in2_conv1", onnx.TensorProto.FLOAT, [29, 1024, 1]
    )
    in2_add1 = onnx.helper.make_tensor_value_info(
        "in2_add1", onnx.TensorProto.FLOAT, [1, 29, 1]
    )
    in2_mul2 = onnx.helper.make_tensor_value_info(
        "in2_mul2", onnx.TensorProto.FLOAT, [1]
    )

    list_of_nodes = [Mul1_node,
            Conv1_node,
            Add1_node,
            Mul2_node,
            Transpose1_node,
            LogSoftmax1_node,
            ArgMax1_node]
    list_of_value_infos = [
            out1_mul1,
            out1_conv1,
            out1_add1,
            out1_mul2,
            out1_transpose1,
            out1_logsoftmax1,
            in2_mul1,
            in2_conv1,
            in2_add1,
            in2_mul2
        ]
    
    if invalid is True:
        list_of_nodes.insert(2, Relu1_node)
        list_of_value_infos.append(out1_relu1)
    
    graph = onnx.helper.make_graph(
            nodes=list_of_nodes,
            name="4d_conversion_test_graph",
            inputs=[in1_mul1],
            outputs=[out1_argmax1],
            value_info=list_of_value_infos
        )        
    onnx_model = onnx.helper.make_model(
        graph, producer_name="4d_conversion_test-model"
    )
    model = ModelWrapper(onnx_model)
    
    return model


def test_4d_conversion():
    """
    Test for the 3D to 4D transformation with a valid graph.
    """
    model = create_arbitrary_model(invalid=False)

    # Inputs
    input_dict = generate_random_input(model)

    # Initializers
    set_all_initializers(model)

    # Comparing the outputs of the model before and after the transform
    output_node_name = model.graph.output[0].name
    output_dict = oxe.execute_onnx(model, input_dict, return_full_exec_context=True)
    expected = output_dict[output_node_name]

    model = model.transform(Change3DTo4DTensors())

    for k, v in input_dict.items():
        old_in_name = k
        old_shape = np.shape(v)
        new_in_name = model.graph.input[0].name
        new_shape = old_shape + (1,)
    new_in_val = np.reshape(v, new_shape)
    del input_dict[old_in_name]
    input_dict[new_in_name] = new_in_val

    output_node_name = model.graph.output[0].name
    output_dict = oxe.execute_onnx(model, input_dict, return_full_exec_context=True)
    expected_modified = output_dict[output_node_name]

    expected_modified = np.reshape(expected_modified, np.shape(expected))

    assert (expected == expected_modified).all()


def test_4d_conversion_invalid_nodes():
    """
    Test for the 3D to 4D transformation when an invalid graph is supplied.
    """
    model = create_arbitrary_model(invalid=True)

    # Inputs
    input_dict = generate_random_input(model)

    # Initializers
    set_all_initializers(model)

    # Comparing the outputs of the model before and after the transform
    output_node_name = model.graph.output[0].name
    output_dict = oxe.execute_onnx(model, input_dict, return_full_exec_context=True)
    expected = output_dict[output_node_name]

    model = model.transform(Change3DTo4DTensors())

    output_node_name = model.graph.output[0].name
    output_dict = oxe.execute_onnx(model, input_dict, return_full_exec_context=True)
    expected_modified = output_dict[output_node_name]

    expected_modified = np.reshape(expected_modified, np.shape(expected))

    assert (expected == expected_modified).all()


In [29]:
test_4d_conversion()
test_4d_conversion_invalid_nodes()

/workspace/finn-base/src/finn/transformation/change_3d_tensors_to_4d.py:82: UserWarning: Found invalid nodes in the graph: ['Relu']. Transformation has not been applied.
  invalid_nodes


In [ ]:
model.save("/tmp/4d_conversion_test_model.onnx")
model = ModelWrapper("/tmp/4d_conversion_test_model.onnx")


# Old code commented (ignore)

In [ ]:
# def convert_3d_to_4d_tensors(model):
#     """ Converts 3D tensors (input, value_info, output) to 4D tensors """
#     tensor_names = {}
#     ## Inputs
#     for t in model.graph.input:
#         tensor_name = t.name
#         tensor_type = t.type.tensor_type.elem_type
#         tensor_shape = model.get_tensor_shape(tensor_name)
#         tensor_names[tensor_name] = [tensor_type]
#         tensor_names[tensor_name].append(tensor_shape)
#
#     ## Initializers
#     initializer_names = []
#     for i in model.graph.initializer:
#         initializer_names.append(i.name)
#
#     ## Value infos
#     for t in model.graph.value_info:
#         tensor_name = t.name
#         if tensor_name in initializer_names:
#             continue
#         else:
#             tensor_type = t.type.tensor_type.elem_type
#             tensor_shape = model.get_tensor_shape(tensor_name)
#             tensor_names[tensor_name] = [tensor_type]
#             tensor_names[tensor_name].append(tensor_shape)
#
#     ## Outputs
#     for t in model.graph.output:
#         tensor_name = t.name
#         tensor_type = t.type.tensor_type.elem_type
#         tensor_shape = model.get_tensor_shape(tensor_name)
#         tensor_names[tensor_name] = [tensor_type]
#         tensor_names[tensor_name].append(tensor_shape)
#
#     ## Find tensors that are the output of nodes that reduce the dimension
#     tensors_reduced_dimension = []
#     for n in model.graph.node:
#         if n.op_type=="ArgMax":
#             keep_dims = get_by_name(n.attribute, "keepdims", "name").i
#             if keep_dims == 0:
#                 node_out = n.output
#                 for n_o in node_out:
#                     tensors_reduced_dimension.append(n_o)
#
#     ## Change format of each value_info + input + output tensors
#     for k, v in tensor_names.items():
#         tensor_type = v[0]
#         shape = v[1]
#         ## Add extra dimension for tensors:
#         # 1) Have 3 dimensions (NCH -> NCH1)
#         # 2) Tensors following operations that reduce their dimension: {Argmax, ...}
#         if len(shape)==3 or k in tensors_reduced_dimension:
#             shape.append(1)
#             model.set_tensor_shape(k, shape, tensor_type)
#         else:
#             continue
#
# def make_node_4d_compatible(model):
#     """ Each node from the list of supported nodes is made compatible with 4D tensors"""
#     for n in model.graph.node:
#         node_op_type = n.op_type
#         if node_op_type == "Transpose":
#             perm = get_by_name(n.attribute,"perm", "name").ints
#             if len(perm) == 3: # Meaning transpose was on 3D tensor
#                 perm.append(3) # append 4th dimension
#         elif node_op_type == "ArgMax" or node_op_type == "LogSoftMax":
#             axis = get_by_name(n.attribute, "axis", "name")
#             if axis.i == -1:
#                 axis.i = 2 # argmax is now on the second-to-last axis
#         elif node_op_type == "Conv":
#             dilations = get_by_name(n.attribute, "dilations", "name").ints
#             kernel_shape = get_by_name(n.attribute, "kernel_shape", "name").ints
#             pads = get_by_name(n.attribute, "pads", "name").ints
#             strides = get_by_name(n.attribute, "strides", "name").ints
#             if len(dilations) == 1: # we must add another dimension to it
#                 dilations.append(dilations[0]) # only equal dilation value along each spatial axis is supported
#             if len(kernel_shape) == 1: # we must add another dimension to it
#                 kernel_shape.append(1)
#             if len(pads) == 2: # pads = ([x1_begin, x1_end] -->) [x1_begin, x2_begin, x1_end, x2_end]
#                 pads.insert(1, 0)
#                 pads.append(0)
#             if len(strides) == 1: # strides = [stride_H, stride_W]
#                 strides.append(1)
#
# def create_4d_initializers_conv_mul_add_node(model):
#     """ Conv, Mul and Add nodes are made compatible with 4D input tensors """
#     initializers = {}
#     for i in model.graph.initializer:
#         init_name = i.name
#         if "Conv" in init_name:
#             init_dim = i.dims
#             init_dtype = i.data_type
#             initializers[init_name] = [init_dtype]
#             initializers[init_name].append(init_dim)
#         elif init_name[0:4] == "Mul_":
#             init_dim = i.dims
#             if len(i.dims) == 3:
#                 init_dtype = i.data_type
#                 initializers[init_name] = [init_dtype]
#                 initializers[init_name].append(init_dim)
#         elif "Add" in init_name:
#             init_dim = i.dims
#             if len(i.dims) == 3:
#                 init_dtype = i.data_type
#                 initializers[init_name] = [init_dtype]
#                 initializers[init_name].append(init_dim)
#         else:
#             continue
#
#     for k, v in initializers.items():
#         init_dtype = v[0]
#         init_shape = v[1]
#         if len(init_shape) == 3:
#             # Change shape NCH -> NCH1
#             init_shape.append(1)
#             shape_init = model.get_initializer(k).shape
#             model.set_tensor_shape(k, init_shape, init_dtype)